<font size=6> The total sovereign emissions calculated by adding up scope 1, 2 and 3 emissions  </font>

In [1]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3
# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
%pip install ipython --upgrade
%pip install country_converter --upgrade
%pip install pint-pandas
%pip install openscm-units
%pip install pint
%pip install jedi
%pip install pycountry

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

#trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3
import pandas as pd

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_total_sovereign_emissions'
src_table_1 = 'sf_oecd_imgr_fco2'
src_table_2 = 'sf_unfccc_results'
src_table_3 = 'sf_oecd_exgr_fco2'


In [6]:
import pandas as pd
from openscm_units import unit_registry
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
# First we create the registry.
ureg = unit_registry
Q_ = ureg.Quantity
ureg.default_format = '~'
ureg.define("CO2e = CO2 = CO2eq")
ureg.define("USD = [currency] ")
ureg.define('Millions=1000000')
set_application_registry(ureg)


In [7]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [8]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [9]:
# Get the values saved in Trino related with Foreign CO2 emissions embodied in gross imports

import pandas as pd

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
df1 = requantify_df(df1)
# convert to CO2 units to t
df1['value'] = df1['value'].pint.to("t CO2e")
df1 = dequantify_df(df1)


sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} where industry_code='DTOTAL'""" 
df_exgr = pd.read_sql(sql, engine)
df_exgr = requantify_df(df_exgr)
# convert to CO2 units to t
df_exgr['value'] = df_exgr['value'].pint.to("t CO2e")
df_exgr = dequantify_df(df_exgr)

#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


<font size=3> Remove invalid country codes </font>

In [10]:
import pycountry
import pandas as pd
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country.rename(columns={'alpha_3':'country_iso_code'})
df_country=df_country[['country_iso_code']]
# add Rest of World to the country dataframe
dict_row = {'country_iso_code':'ROW'}
df_country = df_country.append(dict_row, ignore_index = True)
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])  
df1=df1.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df1

df_exgr = pd.merge(df_exgr,df_country,on=['country_iso_code'])  
df_exgr=df_exgr.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df_exgr






/tmp/ipykernel_407/4216143356.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_country = df_country.append(dict_row, ignore_index = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,ZAF,SVN,DTOTAL,2003,Foreign CO2 emissions embodied in gross exports,3000.0,CO2eq * t
1,ZAF,SVN,DTOTAL,2004,Foreign CO2 emissions embodied in gross exports,3000.0,CO2eq * t
2,ZAF,SVN,DTOTAL,2005,Foreign CO2 emissions embodied in gross exports,6000.0,CO2eq * t
3,ZAF,SVN,DTOTAL,2006,Foreign CO2 emissions embodied in gross exports,10000.0,CO2eq * t
4,ZAF,SVN,DTOTAL,2007,Foreign CO2 emissions embodied in gross exports,9000.0,CO2eq * t
...,...,...,...,...,...,...,...
120787,VNM,MMR,DTOTAL,2014,Foreign CO2 emissions embodied in gross exports,117000.0,CO2eq * t
120788,VNM,MMR,DTOTAL,2015,Foreign CO2 emissions embodied in gross exports,159000.0,CO2eq * t
120789,VNM,MMR,DTOTAL,2016,Foreign CO2 emissions embodied in gross exports,261000.0,CO2eq * t
120790,VNM,MMR,DTOTAL,2017,Foreign CO2 emissions embodied in gross exports,263000.0,CO2eq * t


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [11]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
df1

######################

df_exgr = df_exgr.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df_exgr.convert_dtypes()

df_exgr.info(verbose=True)
df_exgr



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         2016 non-null   string 
 1   industry_code     2016 non-null   string 
 2   country_iso_code  2016 non-null   string 
 3   validity_date     2016 non-null   Int64  
 4   total             2016 non-null   Float64
 5   total_units    

,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Foreign CO2 emissions embodied in gross exports,DTOTAL,APEC,1995,414810000.0,CO2eq * t
1,Foreign CO2 emissions embodied in gross exports,DTOTAL,APEC,1996,441536000.0,CO2eq * t
2,Foreign CO2 emissions embodied in gross exports,DTOTAL,APEC,1997,472415000.0,CO2eq * t
3,Foreign CO2 emissions embodied in gross exports,DTOTAL,APEC,1998,480209000.0,CO2eq * t
4,Foreign CO2 emissions embodied in gross exports,DTOTAL,APEC,1999,536571000.0,CO2eq * t
...,...,...,...,...,...,...
2011,Foreign CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2014,58850000.0,CO2eq * t
2012,Foreign CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2015,49904000.0,CO2eq * t
2013,Foreign CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2016,43192000.0,CO2eq * t
2014,Foreign CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2017,47417000.0,CO2eq * t


In [12]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

df_result = pd.merge(df_result,df_exgr,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total':'exported_emissions','total_units':'exported_emissions_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result.drop(columns=['attribute','industry_code'],inplace=True)
df_result



,country_iso_code,validity_date,scope2_value,scope2_value_units,scope3_value,scope3_value_units,exported_emissions,exported_emissions_units
0,APEC,1995,39478000.0,CO2eq * t,2716610000.0,CO2eq * t,414810000.0,CO2eq * t
1,APEC,1996,39637000.0,CO2eq * t,2687546000.0,CO2eq * t,441536000.0,CO2eq * t
2,APEC,1997,46108000.0,CO2eq * t,2805099000.0,CO2eq * t,472415000.0,CO2eq * t
3,APEC,1998,47416000.0,CO2eq * t,3011055000.0,CO2eq * t,480209000.0,CO2eq * t
4,APEC,1999,47128000.0,CO2eq * t,3218345000.0,CO2eq * t,536571000.0,CO2eq * t
...,...,...,...,...,...,...,...,...
2011,ZSCA,2014,917000.0,CO2eq * t,208345000.0,CO2eq * t,58850000.0,CO2eq * t
2012,ZSCA,2015,619000.0,CO2eq * t,214100000.0,CO2eq * t,49904000.0,CO2eq * t
2013,ZSCA,2016,566000.0,CO2eq * t,205862000.0,CO2eq * t,43192000.0,CO2eq * t
2014,ZSCA,2017,433000.0,CO2eq * t,209461000.0,CO2eq * t,47417000.0,CO2eq * t


In [13]:
# Get the values saved in Trino containing  GHG emissions provided by UNFCCC & GDP PPP values from Worldbank

import pandas as pd

sql=f"""
select country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,ghg_total_with_lulucf,ghg_total_with_lulucf_units
,gdp,gdp_units,gdp_ppp,gdp_ppp_units
from {ingest_catalog}.{ingest_schema}.{src_table_2} """ 
df_unfccc = pd.read_sql(sql, engine)
df_unfccc[df_unfccc['country_iso_code']=='ARG']
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


,country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,ghg_total_with_lulucf,ghg_total_with_lulucf_units,gdp,gdp_units,gdp_ppp,gdp_ppp_units
1012,ARG,1992,NaN,CO2eq * kt,NaN,CO2eq * kt,2.287886e+11,USD,2.921593e+11,USD
1013,ARG,1998,NaN,CO2eq * kt,NaN,CO2eq * kt,2.989482e+11,USD,4.309373e+11,USD
1014,ARG,2018,NaN,CO2eq * kt,NaN,CO2eq * kt,5.248197e+11,USD,1.036428e+12,USD
1015,ARG,2009,NaN,CO2eq * kt,NaN,CO2eq * kt,3.329765e+11,USD,6.612758e+11,USD
1961,ARG,2014,NaN,CO2eq * kt,NaN,CO2eq * kt,5.263197e+11,USD,8.398967e+11,USD
1962,ARG,2007,NaN,CO2eq * kt,NaN,CO2eq * kt,2.875305e+11,USD,6.575692e+11,USD
1963,ARG,1996,NaN,CO2eq * kt,NaN,CO2eq * kt,2.721498e+11,USD,3.728882e+11,USD
1964,ARG,1999,NaN,CO2eq * kt,NaN,CO2eq * kt,2.835230e+11,USD,4.223593e+11,USD
3165,ARG,2005,NaN,CO2eq * kt,NaN,CO2eq * kt,1.987371e+11,USD,5.277291e+11,USD
3166,ARG,2017,NaN,CO2eq * kt,NaN,CO2eq * kt,6.436287e+11,USD,1.039331e+12,USD


In [14]:
import numpy as np
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)

df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))

df_unfccc = requantify_df(df_unfccc)
df_unfccc.info(verbose=True)
# convert to CO2 units to Mt
df_unfccc['ghg_total_without_lulucf'] = df_unfccc['ghg_total_without_lulucf'].pint.to("t CO2e")
df_unfccc['ghg_total_with_lulucf'] = df_unfccc['ghg_total_with_lulucf'].pint.to("t CO2e")
####

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf==0, np.nan, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf==0, np.nan, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp==0, np.nan, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp==0, np.nan, df_unfccc.gdp_ppp))

df_unfccc.info(verbose=True)


df_unfccc[df_unfccc['country_iso_code']=='ARG']

#df_unfccc['gdp'] = df_unfccc['gdp'].pint.to("Millions USD")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7185 entries, 0 to 7184
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                7185 non-null   string 
 1   validity_date                   7185 non-null   Int64  
 2   ghg_total_without_lulucf        2042 non-null   Float64
 3   ghg_total_without_lulucf_units  7185 non-null   string 
 4   ghg_total_with_lulucf           2042 non-null   Float64
 5   ghg_total_with_lulucf_units     7185 non-null   string 
 6   gdp                             7185 non-null   Float64
 7   gdp_units                       7185 non-null   string 
 8   gdp_ppp                         7185 non-null   Float64
 9   gdp_ppp_units                   7185 non-null   string 
dtypes: Float64(4), Int64(1), string(5)
memory usage: 596.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7185 entries, 0 to 7184
Data columns (total 6 co

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_with_lulucf,gdp,gdp_ppp
1012,ARG,1992,0.0,0.0,228788617201.696,292159264258.284
1013,ARG,1998,0.0,0.0,298948250000.0,430937274294.631
1014,ARG,2018,0.0,0.0,524819742918.669,1036428172096.8
1015,ARG,2009,0.0,0.0,332976484577.619,661275816702.498
1961,ARG,2014,0.0,0.0,526319673731.638,839896688260.086
1962,ARG,2007,0.0,0.0,287530508430.568,657569211584.856
1963,ARG,1996,0.0,0.0,272149750000.0,372888233736.503
1964,ARG,1999,0.0,0.0,283523000000.0,422359326228.804
3165,ARG,2005,0.0,0.0,198737095012.282,527729109507.491
3166,ARG,2017,0.0,0.0,643628665302.155,1039330591568.54


In [15]:
#df_unfccc = dequantify_df(df_unfccc)
#df_unfccc
#df_unfccc.info(verbose=True)
#df_unfccc

#df_result= requantify_df(df_result)
#df_result= dequantify_df(df_result)
#df_result


<font size=3><b>Merge UNFCCC , Worldbank and OECD data</b> </font>

In [16]:
df_result.info(verbose=True)
df_result= requantify_df(df_result)
df_result.info(verbose=True)

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))


df_result = pd.merge(df_result,df_unfccc,on=['country_iso_code','validity_date'],how='outer')  

rename_columns = {'ghg_total_without_lulucf':'scope1_excl_lulucf',
                  'ghg_total_with_lulucf':'scope1_incl_lulucf'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)

df_result['sum_yn'] = 1 * ((df_result['scope1_excl_lulucf'] > 0) & (df_result['scope2_value'] > 0) &  (df_result['scope3_value'] > 0) )

df_result['consumption_emissions_excl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])
df_result['consumption_emissions_incl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])
###############
df_result['attribution_factor_scope1_excl_lulucf']= (df_result['scope1_excl_lulucf']) / df_result['gdp_ppp']
df_result['attribution_factor_scope1_incl_lulucf']= (df_result['scope1_incl_lulucf']) / df_result['gdp_ppp']

df_result['attribution_factor_excl_lulucf']= df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
df_result['attribution_factor_incl_lulucf']= df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
##########
df_result['attribution_factor_excl_lulucf_gdp']= df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])
df_result['attribution_factor_incl_lulucf_gdp']= df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])



#df_result['ghg_total_without_lulucf']=df_result['ghg_total_without_lulucf'].replace(0, np.nan)
#df_result['ghg_total_with_lulucf']=df_result['ghg_total_with_lulucf'].replace(0, np.nan)
#df_result['gdp']=df_result['gdp'].replace(0, np.nan)
#df_result['gdp_ppp']=df_result['gdp_ppp'].replace(0, np.nan)



#df_result

columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result = df_result.reindex(columns=columns_order)
df_result.info(verbose=True)
df_result= dequantify_df(df_result)
df_result.info(verbose=True)
# remove invalid country_iso_codes (regions, ...)

df_result = pd.merge(df_result,df_country,on=['country_iso_code'])  

cols= ['scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result[cols] = df_result[cols].replace({0:np.nan})


df_result


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country_iso_code          2016 non-null   string 
 1   validity_date             2016 non-null   Int64  
 2   scope2_value              2016 non-null   Float64
 3   scope2_value_units        2016 non-null   string 
 4   scope3_value              2016 non-null   Float64
 5   scope3_value_units        2016 non-null   string 
 6   exported_emissions        2016 non-null   Float64
 7   exported_emissions_units  2016 non-null   string 
dtypes: Float64(3), Int64(1), string(4)
memory usage: 149.6 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 2015
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   country_iso_code    2016 non-null   string         
 1  

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
0,ARG,1995,NaN,CO2eq * t,NaN,CO2eq * t,21000.0,CO2eq * t,9356000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
1,ARG,1996,NaN,CO2eq * t,NaN,CO2eq * t,28000.0,CO2eq * t,11287000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
2,ARG,1997,2.709103e+08,CO2eq * t,2.419562e+08,CO2eq * t,26000.0,CO2eq * t,14968000.0,CO2eq * t,...,0.00059,CO2eq * t / USD,0.000697,CO2eq * t / USD,0.000976,CO2eq * t / USD,0.000626,CO2eq * t / USD,0.000877,CO2eq * t / USD
3,ARG,1998,NaN,CO2eq * t,NaN,CO2eq * t,43000.0,CO2eq * t,13397000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
4,ARG,1999,NaN,CO2eq * t,NaN,CO2eq * t,61000.0,CO2eq * t,12270000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5749,GRD,2020,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
5750,GRD,2012,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
5751,GRD,2016,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
5752,GRD,2001,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD


In [17]:
#df_result['scope1_value_units'] = 'Mt CO2e'
#df_result['scope1_value'] = df_result['scope1_value'].round(decimals = 3)
#pd.options.display.float_format = '{:.3f}'.format
#df_result= df_result.convert_dtypes()
#df_result.info(verbose=True)
df_result[df_result['country_iso_code']=='CAN']
#df_result

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
217,CAN,1995,NaN,CO2eq * t,NaN,CO2eq * t,5129000.0,CO2eq * t,179111000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
218,CAN,1996,NaN,CO2eq * t,NaN,CO2eq * t,5269000.0,CO2eq * t,188198000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
219,CAN,1997,6.911954e+08,CO2eq * t,6.529372e+08,CO2eq * t,6362000.0,CO2eq * t,202972000.0,CO2eq * t,...,0.000864,CO2eq * t / USD,0.001192,CO2eq * t / USD,0.001375,CO2eq * t / USD,0.001141,CO2eq * t / USD,0.001316,CO2eq * t / USD
220,CAN,1998,6.969015e+08,CO2eq * t,6.470244e+08,CO2eq * t,7993000.0,CO2eq * t,214108000.0,CO2eq * t,...,0.000815,CO2eq * t / USD,0.001158,CO2eq * t / USD,0.001450,CO2eq * t / USD,0.001095,CO2eq * t / USD,0.001371,CO2eq * t / USD
221,CAN,1999,7.096009e+08,CO2eq * t,6.742546e+08,CO2eq * t,9100000.0,CO2eq * t,222868000.0,CO2eq * t,...,0.000797,CO2eq * t / USD,0.001112,CO2eq * t / USD,0.001388,CO2eq * t / USD,0.001071,CO2eq * t / USD,0.001336,CO2eq * t / USD
222,CAN,2000,7.335113e+08,CO2eq * t,7.117703e+08,CO2eq * t,18191000.0,CO2eq * t,236036000.0,CO2eq * t,...,0.000790,CO2eq * t / USD,0.001096,CO2eq * t / USD,0.001326,CO2eq * t / USD,0.001072,CO2eq * t / USD,0.001297,CO2eq * t / USD
223,CAN,2001,7.229350e+08,CO2eq * t,6.852524e+08,CO2eq * t,19531000.0,CO2eq * t,228887000.0,CO2eq * t,...,0.000731,CO2eq * t / USD,0.001036,CO2eq * t / USD,0.001314,CO2eq * t / USD,0.000996,CO2eq * t / USD,0.001263,CO2eq * t / USD
224,CAN,2002,7.265552e+08,CO2eq * t,7.062825e+08,CO2eq * t,8195000.0,CO2eq * t,233500000.0,CO2eq * t,...,0.000727,CO2eq * t / USD,0.000997,CO2eq * t / USD,0.001273,CO2eq * t / USD,0.000976,CO2eq * t / USD,0.001246,CO2eq * t / USD
225,CAN,2003,7.448319e+08,CO2eq * t,7.203933e+08,CO2eq * t,8220000.0,CO2eq * t,232392000.0,CO2eq * t,...,0.000704,CO2eq * t / USD,0.000963,CO2eq * t / USD,0.001100,CO2eq * t / USD,0.000939,CO2eq * t / USD,0.001073,CO2eq * t / USD
226,CAN,2004,7.459198e+08,CO2eq * t,7.493836e+08,CO2eq * t,8411000.0,CO2eq * t,238375000.0,CO2eq * t,...,0.000692,CO2eq * t / USD,0.000916,CO2eq * t / USD,0.000967,CO2eq * t / USD,0.000919,CO2eq * t / USD,0.000970,CO2eq * t / USD


Calculate Attribution factor = (Scope1+Scope2+Scope3)/GDP


In [18]:


#df_result= requantify_df(df_result)
format_mapper =  {'gdp_ppp': '{0:,.0f}',
           'gdp': '{0:,.0f}',       
           'attribution_factor_excl_lulucf': '{0:.8f}',
           'attribution_factor_incl_lulucf': '{0:.8f}',
           'attribution_factor_excl_lulucf_gdp': '{0:.8f}',
           'attribution_factor_incl_lulucf_gdp': '{0:.8f}',       
           'scope1_excl_lulucf': '{0:,.0f}',       
           'scope1_incl_lulucf': '{0:,.0f}',              
           'scope2_value': '{0:,.0f}',              
           'scope3_value': '{0:,.0f}'
                 }

df_result[df_result['country_iso_code']=='CAN'].style.format(format_mapper)


,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,exported_emissions,exported_emissions_units,gdp_ppp,gdp_ppp_units,gdp,gdp_units,consumption_emissions_excl_lulucf,consumption_emissions_excl_lulucf_units,consumption_emissions_incl_lulucf,consumption_emissions_incl_lulucf_units,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_excl_lulucf_units,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
217,CAN,1995,nan,CO2eq * t,nan,CO2eq * t,"5,129,000",CO2eq * t,"179,111,000",CO2eq * t,25706000.000000,CO2eq * t,nan,USD,nan,USD,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
218,CAN,1996,nan,CO2eq * t,nan,CO2eq * t,"5,269,000",CO2eq * t,"188,198,000",CO2eq * t,26163000.000000,CO2eq * t,nan,USD,nan,USD,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
219,CAN,1997,"691,195,367",CO2eq * t,"652,937,162",CO2eq * t,"6,362,000",CO2eq * t,"202,972,000",CO2eq * t,29525000.000000,CO2eq * t,"755,390,170,461",USD,"654,986,999,856",USD,871004367.182711,CO2eq * t,832746162.403060,CO2eq * t,0.000915,CO2eq * t / USD,0.000864,CO2eq * t / USD,0.00119214,CO2eq * t / USD,0.00137488,CO2eq * t / USD,0.00114149,CO2eq * t / USD,0.00131647,CO2eq * t / USD
220,CAN,1998,"696,901,519",CO2eq * t,"647,024,422",CO2eq * t,"7,993,000",CO2eq * t,"214,108,000",CO2eq * t,33586000.000000,CO2eq * t,"793,674,327,704",USD,"634,000,000,000",USD,885416518.588772,CO2eq * t,835539422.222185,CO2eq * t,0.000878,CO2eq * t / USD,0.000815,CO2eq * t / USD,0.00115791,CO2eq * t / USD,0.00144953,CO2eq * t / USD,0.00109507,CO2eq * t / USD,0.00137086,CO2eq * t / USD
221,CAN,1999,"709,600,924",CO2eq * t,"674,254,607",CO2eq * t,"9,100,000",CO2eq * t,"222,868,000",CO2eq * t,36403000.000000,CO2eq * t,"846,412,945,810",USD,"678,412,196,271",USD,905165924.294885,CO2eq * t,869819607.218812,CO2eq * t,0.000838,CO2eq * t / USD,0.000797,CO2eq * t / USD,0.00111242,CO2eq * t / USD,0.00138790,CO2eq * t / USD,0.00107066,CO2eq * t / USD,0.00133580,CO2eq * t / USD
222,CAN,2000,"733,511,337",CO2eq * t,"711,770,284",CO2eq * t,"18,191,000",CO2eq * t,"236,036,000",CO2eq * t,41605000.000000,CO2eq * t,"900,996,986,801",USD,"744,773,415,932",USD,946133337.261223,CO2eq * t,924392284.018432,CO2eq * t,0.000814,CO2eq * t / USD,0.000790,CO2eq * t / USD,0.00109627,CO2eq * t / USD,0.00132623,CO2eq * t / USD,0.00107214,CO2eq * t / USD,0.00129704,CO2eq * t / USD
223,CAN,2001,"722,935,012",CO2eq * t,"685,252,427",CO2eq * t,"19,531,000",CO2eq * t,"228,887,000",CO2eq * t,36026000.000000,CO2eq * t,"937,786,776,177",USD,"738,981,792,355",USD,935327012.117626,CO2eq * t,897644426.599736,CO2eq * t,0.000771,CO2eq * t / USD,0.000731,CO2eq * t / USD,0.00103579,CO2eq * t / USD,0.00131445,CO2eq * t / USD,0.00099561,CO2eq * t / USD,0.00126346,CO2eq * t / USD
224,CAN,2002,"726,555,183",CO2eq * t,"706,282,510",CO2eq * t,"8,195,000",CO2eq * t,"233,500,000",CO2eq * t,36197000.000000,CO2eq * t,"971,003,788,233",USD,"760,649,334,098",USD,932053183.452183,CO2eq * t,911780509.635937,CO2eq * t,0.000748,CO2eq * t / USD,0.000727,CO2eq * t / USD,0.00099716,CO2eq * t / USD,0.00127293,CO2eq * t / USD,0.00097629,CO2eq * t / USD,0.00124627,CO2eq * t / USD
225,CAN,2003,"744,831,887",CO2eq * t,"720,393,316",CO2eq * t,"8,220,000",CO2eq * t,"232,392,000",CO2eq * t,38673000.000000,CO2eq * t,"1,023,682,523,003",USD,"895,540,646,635",USD,946770886.836370,CO2eq * t,922332315.547560,CO2eq * t,0.000728,CO2e

<font size=3>Save the results in Trino</font>

In [19]:
# Add the sources used for the calculations in the result table
df_result['scope1_excl_source'] = "UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope1_incl_source'] = "UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope2_source'] = "OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE"
df_result['scope3_source'] = "OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value "          
df_result['gdp_ppp_source'] = "Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv"          


In [20]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)
columnschema = osc.create_table_schema_pairs(df_result) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5754 entries, 0 to 5753
Data columns (total 37 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   country_iso_code                             5754 non-null   string 
 1   validity_date                                5754 non-null   Int64  
 2   scope1_excl_lulucf                           2042 non-null   Float64
 3   scope1_excl_lulucf_units                     5754 non-null   string 
 4   scope1_incl_lulucf                           2042 non-null   Float64
 5   scope1_incl_lulucf_units                     5754 non-null   string 
 6   scope2_value                                 1419 non-null   Float64
 7   scope2_value_units                           5754 non-null   string 
 8   scope3_value                                 1608 non-null   Int64  
 9   scope3_value_units                           5754 non-null   string 
 10  

In [21]:
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_total_sovereign_emissions"
inserting 1000 records
  ('ARG', 1995, NULL, 'CO2eq * t', NULL, 'CO2eq * t', 21000.0, 'CO2eq * t', 9356000, 'CO2eq * t', 3312000, 'CO2eq * t', 347005607350.473, 'USD', 258031750000.0, 'USD', NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', 'UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx', 'UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx', 'OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE', 'OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value ', 'Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv')
  ('ARG', 1996, NULL, 'CO2eq * t', NULL, 'CO2eq * t', 28000.0, 'CO2eq * t', 11287000, 'CO2eq * t', 3809000, 'CO2eq * t', 372888233736.503, 'USD', 2721497

In [22]:
df_result.sort_values(by=['country_iso_code','validity_date'], inplace=True)
df_result.to_csv("results_2.csv",index=False,encoding='utf-8')
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table} where country_iso_code='AUT'"""
pd.read_sql(sql, engine)

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units,scope1_excl_source,scope1_incl_source,scope2_source,scope3_source,gdp_ppp_source
0,AUT,1995,7.923798e+07,CO2eq * t,6.586448e+07,CO2eq * t,689000.0,CO2eq * t,28359000.0,CO2eq * t,...,CO2eq * t / USD,0.000504,CO2eq * t / USD,0.000394,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
1,AUT,1996,8.245259e+07,CO2eq * t,7.172106e+07,CO2eq * t,766000.0,CO2eq * t,29358000.0,CO2eq * t,...,CO2eq * t / USD,0.000521,CO2eq * t / USD,0.000429,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
2,AUT,1997,8.211623e+07,CO2eq * t,6.287460e+07,CO2eq * t,794000.0,CO2eq * t,32055000.0,CO2eq * t,...,CO2eq * t / USD,0.000472,CO2eq * t / USD,0.000450,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
3,AUT,1998,8.143285e+07,CO2eq * t,6.402936e+07,CO2eq * t,762000.0,CO2eq * t,32710000.0,CO2eq * t,...,CO2eq * t / USD,0.000458,CO2eq * t / USD,0.000447,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
4,AUT,1999,7.989837e+07,CO2eq * t,6.019217e+07,CO2eq * t,774000.0,CO2eq * t,33134000.0,CO2eq * t,...,CO2eq * t / USD,0.000426,CO2eq * t / USD,0.000433,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
5,AUT,2000,8.012943e+07,CO2eq * t,6.350221e+07,CO2eq * t,973000.0,CO2eq * t,36172000.0,CO2eq * t,...,CO2eq * t / USD,0.000428,CO2eq * t / USD,0.000510,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
6,AUT,2001,8.406472e+07,CO2eq * t,6.459651e+07,CO2eq * t,1630000.0,CO2eq * t,37747000.0,CO2eq * t,...,CO2eq * t / USD,0.000435,CO2eq * t / USD,0.000526,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
7,AUT,2002,8.581506e+07,CO2eq * t,7.138788e+07,CO2eq * t,2243000.0,CO2eq * t,38112000.0,CO2eq * t,...,CO2eq * t / USD,0.000443,CO2eq * t / USD,0.000521,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...
8,AUT,2003,9.130672e+07,CO2eq * t,8.627772e+07,CO2eq * t,2893000.0,CO2eq * t,39849000.0,CO2eq * t,...,CO2eq * t / USD,0.000494,CO2eq * t / USD,0.000492,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF..